<a href="https://colab.research.google.com/github/vilkaste/compling_course33/blob/main/%D0%9A%D1%83%D0%B7%D0%BD%D0%B5%D1%86%D0%BE%D0%B2%D0%B0_%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [3]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [4]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

1.   word2vec-ruscorpora-300 (НКРЯ, 184973 векторов; 250 млн словоупотреблений; 300 размерность, для русского языка и работы с русскоязычным маетриалом и текстом)
2.   glove-twitter-100 (Твиттер, 1193514 векторов; 27 млн токенов; размерность 200, для работы с современным языком и трендами, "повесткой")
3. word2vec-google-news-300 (3000000 векторов; 100 миллионов словоупотреблений; размерность 200, подойдет для работы с более нейтральными текстами, однако большой охват, как мне кажется, так как новостизатрагивают большое количество разных полей)



**Базовые операции с векторами**

In [5]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [6]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [7]:
# загрузка модели
w2v_model_2 = api.load("word2vec-google-news-300")

# вывод размера словаря (число слов)
print(f"Размер словаря: {len(w2v_model_2.key_to_index)}")

# вывод размерности векторов
print(f"Размерность векторов: {w2v_model_2.vector_size}")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Размер словаря: 3000000
Размерность векторов: 300


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [34]:
target_word = input('Введите слово для получения 10 слов близких по вектору: ')

similar_words = w2v_model_2.most_similar(target_word, topn=10)

print("близкие слова")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Введите слово для получения 10 слов близких по вектору: christianity
близкие слова
  christian: 0.6904
  christians: 0.6683
  islam: 0.6658
  religon: 0.6436
  Christianity: 0.6338
  athiests: 0.6187
  mormons: 0.6159
  athiest: 0.6125
  catholicism: 0.6079
  relgion: 0.6044


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [35]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [38]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,   # размер вектора
    window=3,         # размер окна
    min_count=1,      # минимальная частота слова
    workers=2,        # кол-во потоков = 2
    sg=1,             # используем Skip-Gram
)

print(f"Размер словаря: {len(sg_model.wv.key_to_index)}")

Размер словаря: 65


In [39]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [40]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [44]:
# Найдите слова, похожие на "духовка"
### ваш код здесь ###

# ищем слова
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"
### ваш код здесь ###

# ищем слова
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [46]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [49]:
# ищем слова
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

    # ищем слова
try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# ищем слова
try:
    similar = ft_model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")



Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [52]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('лемон')
compare_models('мисить')
compare_models('жарит')


Сравнение для слова: 'лемон'
  Word2Vec: слово не найдено
  FastText: ['сковорода', 'масло']

Сравнение для слова: 'мисить'
  Word2Vec: слово не найдено
  FastText: ['начинка', 'бекон']

Сравнение для слова: 'жарит'
  Word2Vec: слово не найдено
  FastText: ['жарить', 'пирог']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [53]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [54]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [55]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [56]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [58]:
# сравниваем схожесть документов
def compare_documents(doc2_id, doc4_id):

    similarity = doc_model.dv.similarity(f"doc_{doc2_id}", f"doc_{doc4_id}")
    print(f"Схожесть doc_{doc2_id} и doc_{doc4_id}: {similarity:.4f}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")
    print(f"  doc_{doc4_id}: {documents[doc4_id]}")

compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [59]:
doc_vector = doc_model.dv["doc_1"]
print(f"Вектор документа doc_1: {doc_vector[:5]}...")

similar_docs = doc_model.dv.most_similar("doc_1", topn=2)
print("\nДокументы, похожие на doc_1:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_1: [-0.00435809 -0.00022557 -0.01330682 -0.01307992 -0.00394024]...

Документы, похожие на doc_1:
  doc_0: 0.2735
    Текст: machine learning is interesting
  doc_3: 0.2031
    Текст: artificial intelligence is amazing


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [61]:

# обучаем модель FastText с размерностью 10
ft_model_10 = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2
)

# обучаем модель FastText с размерностью 50
ft_model_50 = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

# обучаем модель FastText с размерностью 100
ft_model_100 = FastText(
    sentences=cooking_sentences,
    vector_size=100,
    window=3,
    min_count=1,
    workers=2
)

test_words = ['вино', 'кастрюля', 'кинза'] # слово есть, нет, нет. для проверки модели

def comparing(model, model_name, test_words):
    print(f"\nМодель: {model_name} (vector_size={model.vector_size})")

    for word in test_words:
        try:
            similar_words = model.wv.most_similar(word, topn=5)
            print(f"\nСлова, похожие на '{word}':")
            for similar_word, score in similar_words:
                print(f"  {similar_word}: {score:.4f}")
        except KeyError:
            print(f"\nСлово '{word}' не найдено в словаре")

# сравниваем модель
comparing(ft_model_10, "FastText 10", test_words)
comparing(ft_model_50, "FastText 50", test_words)
comparing(ft_model_100, "FastText 100", test_words)


Модель: FastText 10 (vector_size=10)

Слова, похожие на 'вино':
  жарить: 0.5608
  парить: 0.5389
  яблоки: 0.5317
  мука: 0.4631
  здоровое: 0.4416

Слова, похожие на 'кастрюля':
  барбекю: 0.5832
  сковорода: 0.5718
  месить: 0.5516
  сливки: 0.5101
  уголь: 0.4468

Слова, похожие на 'кинза':
  травы: 0.7057
  соус: 0.6498
  печь: 0.5775
  чашка: 0.5654
  яичница: 0.4880

Модель: FastText 50 (vector_size=50)

Слова, похожие на 'вино':
  месить: 0.2792
  дрожжи: 0.2585
  картофель: 0.2583
  яйца: 0.2365
  гриль: 0.2046

Слова, похожие на 'кастрюля':
  лимон: 0.3262
  запекать: 0.1596
  яйца: 0.1455
  хлеб: 0.1445
  ингредиенты: 0.1417

Слова, похожие на 'кинза':
  мариновать: 0.2590
  кипятить: 0.2547
  овощи: 0.2312
  пирог: 0.1971
  вода: 0.1288

Модель: FastText 100 (vector_size=100)

Слова, похожие на 'вино':
  соль: 0.2365
  яйца: 0.1988
  салат: 0.1898
  курица: 0.1821
  запекать: 0.1497

Слова, похожие на 'кастрюля':
  яичница: 0.3324
  торт: 0.2023
  вино: 0.1752
  дрожжи: 0.